In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
def format_number(x):
    if isinstance(x, float):
        if x.is_integer():
            return int(x)  # Convert to integer if the float is a whole number
        else:
            return '{:.2f}'.format(x).rstrip('0').rstrip('.')  # Keep up to 2 decimals, remove trailing zeros
    return x

def flatten_json(json_data):
    data = []
    for circuit, circuit_value in json_data.items(): # circuit size
        for benchmark, benchmark_value in circuit_value.items(): # benchmark_name
            for layout, metrics in benchmark_value.items():
                if layout == 'init':
                    continue
                # if 'lookahead_swap' not in metrics:
                #     continue
                basic_swap = metrics.get('basic_swap', np.nan)
                sabre_swap = metrics.get('sabre_swap', np.nan)
                lookahead_swap = metrics.get('lookahead_swap', np.nan)
# Percentage Reduction=( Swap Sabre−Swap Lookahead) / Swap Sabre * 100
                basic_swap_reduction = ((basic_swap - lookahead_swap) / basic_swap * 100) if basic_swap != 0 and basic_swap is not np.nan and lookahead_swap is not np.nan else np.nan
                sabre_swap_reduction = ((sabre_swap - lookahead_swap) / sabre_swap * 100) if sabre_swap != 0 and sabre_swap is not np.nan and lookahead_swap is not np.nan else np.nan
                
                basic_depth = metrics.get('basic_depth', np.nan)
                sabre_depth = metrics.get('sabre_depth', np.nan)
                lookahead_depth = metrics.get('lookahead_depth', np.nan)
                basic_depth_reduction = ((basic_depth - lookahead_depth) / basic_depth * 100) if basic_depth != 0 and basic_depth is not np.nan and lookahead_depth is not np.nan else np.nan
                sabre_depth_reduction = ((sabre_depth - lookahead_depth) / sabre_depth * 100) if sabre_depth != 0 and sabre_depth is not np.nan and lookahead_depth is not np.nan else np.nan

                data.append({
                    "benchmark": benchmark,
                    "size": circuit, # circuit size
                    "gate": benchmark_value['init']['size'], # total gates
                    "depth": benchmark_value['init']['depth'], # circuit depth
                    "layout": layout,
                    "swap basic": basic_swap, # basic swap
                    "swap sabre": sabre_swap, # sabre swap
                    "swap lookahead": lookahead_swap, # lookahead swap
                    "swap delta basic": basic_swap_reduction,
                    "swap delta sabre": sabre_swap_reduction, # difference between sabre and lookahead, if positive then reduced but if negative then larger than sabre
                    "depth basic": basic_depth, # basic depth
                    "depth sabre": sabre_depth, # sabre depth
                    "depth lookahead": lookahead_depth, # lookahead depth
                    "depth delta basic": basic_depth_reduction,
                    "depth delta sabre": sabre_depth_reduction
                })
    return pd.DataFrame(data)


In [3]:
def rearrange_columns(df, first_col_name):
    """
    Reorder the columns of the DataFrame so that the specified column is first.
    
    Parameters:
    - df: The input DataFrame.
    - first_col_name: The name of the column to move to the first position.
    
    Returns:
    - A DataFrame with the specified column moved to the first position.
    """
    # Check if the specified column exists in the DataFrame
    if first_col_name not in df.columns:
        raise ValueError(f"Column '{first_col_name}' does not exist in the DataFrame.")
    
    # Create a list of columns with the specified column first
    columns = [first_col_name] + [col for col in df.columns if col != first_col_name]
    
    # Reorder the DataFrame
    df_reordered = df[columns]
    
    return df_reordered

In [4]:
def open_json_from_file():
    with open("result/benchmarking_FINAL.json") as json_file:
        return json.load(json_file)

dict_benchmark = open_json_from_file()

$\Delta s_B$

In [5]:
df = flatten_json(dict_benchmark)
df = df.map(format_number)

def rename_columns(df):
    df.columns = [
        "benchmark",
        # "n",
        "g",
        "d",
        "layout",
        r"$s_B$", # basic swap
        r"$s_S$", # sabre swap
        r"$s_L$", # lookahead swap
        # "swap (%)",
        r"$\Delta s_B$", # swap basic reduction
        r"$\Delta s_S$", # swap sabre reduction
        r"$d_B$", # basic depth
        r"$d_S$", # sabre depth
        r"$d_L$", # lookahead depth
        # "d (%)",
        r"$\Delta d_B$",
        r"$\Delta d_S$"
    ]
    return df
# df = rename_columns(df)

print(df)

    benchmark size  gate  depth            layout  swap basic  swap sabre  \
0          dj    5    36     11         full_20_1           0           0   
1          dj    5    36     11         line_20_1          36           6   
2          dj    5    36     11         full_10_2           0           3   
3          dj    5    36     11          full_7_3           0           0   
4          dj    5    36     11          grid_9_2           9           3   
..        ...  ...   ...    ...               ...         ...         ...   
580    wstate   15   253    135  t_horizontal_5_4          63           3   
581    wstate   15   253    135    t_vertical_5_4         126          15   
582    wstate   15   253    135          ring_5_4           0           3   
583    wstate   15   253    135          full_5_4           0           0   
584    wstate   15   253    135          grid_6_4          21           0   

    swap lookahead swap delta basic swap delta sabre  depth basic  \
0     

In [10]:
def save_latex_to_file(latex, circuit_size, filename=''):
    with open(f"latex/data_{filename}_{circuit_size}_FINAL.tex", "w") as file:
        file.write(latex)

In [12]:
def build_by_benchmark(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.replace('_', '\\_', regex=True)
    df = df.sort_values(by=['gate', 'benchmark'])
    df = df.drop(columns=['size'])
    df = rename_columns(df)
    latex = df.to_latex(index=False, escape=False, longtable=True, caption=f"Additional swap gates and circuit depth, n = {circuit_size}", label=f"benchmark-table-{circuit_size}")
    save_latex_to_file(latex, circuit_size, 'benchmark')
    return df

In [13]:
import os

def build_by_benchmark_excel(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.sort_values(by=['gate', 'benchmark'])
    with pd.ExcelWriter("excel/data_benchmark.xlsx", mode="a" if os.path.exists("excel/data_benchmark.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
        df.to_excel(writer, sheet_name=f"n = {circuit_size}", index=False)
    return df

In [18]:
df_benchmark_5 = build_by_benchmark(df, 5)
df_benchmark_10 = build_by_benchmark(df, 10)
df_benchmark_15 = build_by_benchmark(df, 15)

In [20]:
build_by_benchmark_excel(df, 15)
build_by_benchmark_excel(df, 10)
build_by_benchmark_excel(df, 5)

,benchmark,size,gate,depth,layout,swap basic,swap sabre,swap lookahead,swap delta basic,swap delta sabre,depth basic,depth sabre,depth lookahead,depth delta basic,depth delta sabre
13,ghz,5,7,7,full_20_1,0,0,0,nan,nan,7,7,7,0,0
14,ghz,5,7,7,line_20_1,0,0,18,nan,nan,7,7,9,-28.57,-28.57
15,ghz,5,7,7,full_10_2,0,0,0,nan,nan,7,7,7,0,0
16,ghz,5,7,7,full_7_3,0,0,0,nan,nan,7,7,7,0,0
17,ghz,5,7,7,grid_9_2,6,0,6,0,nan,13,7,8,38.46,-14.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,portfoliovqe,5,310,107,t_horizontal_5_4,117,69,93,20.51,-34.78,239,187,125,47.7,33.16
61,portfoliovqe,5,310,107,t_vertical_5_4,117,48,93,20.51,-93.75,239,193,125,47.7,35.23
62,portfoliovqe,5,310,107,ring_5_4,57,45,93,-63.16,-106.67,146,158,125,14.38,20.89
63,portfoliovqe,5,310,107,full_5_4,0,0,0,nan,nan,107,107,107,0,0


In [21]:
def build_by_layout(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
    df = df.replace('_', '\\_', regex=True)
    df = df.drop(columns=['size']) # remove circuit size
    df = rename_columns(df)
    df = rearrange_columns(df, 'layout')
    latex = df.to_latex(index=False, escape=False, longtable=True, caption=f"Additional swap gates and circuit depth, n = {circuit_size}", label=f"benchmark-table-{circuit_size}")
    save_latex_to_file(latex, circuit_size, 'layout')
    return df

In [22]:
def build_by_layout_excel(df, circuit_size):
    df = df[df['size'] == f"{circuit_size}"]
    df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
    df = rearrange_columns(df, 'layout')

    with pd.ExcelWriter("excel/data_layout.xlsx", mode="a" if os.path.exists("excel/data_layout.xlsx") else "w", engine="openpyxl") as writer: # change mode between a = append and w = write
        df.to_excel(writer, sheet_name=f"n = {circuit_size}", index=False)
    return df

In [23]:
df_layout_5 = build_by_layout(df, 5)
df_layout_10 = build_by_layout(df, 10)
df_layout_15 = build_by_layout(df, 15)

/tmp/ipykernel_25415/2202330249.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_25415/2202330249.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_25415/2202330249.py:3: DeprecationWarning: DataFram

In [24]:
build_by_layout_excel(df, 15)
build_by_layout_excel(df, 10)
build_by_layout_excel(df, 5)

/tmp/ipykernel_25415/3532246934.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_25415/3532246934.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by=['layout']).apply(lambda x: x.sort_values(by=['gate', 'benchmark']))
/tmp/ipykernel_25415/3532246934.py:3: DeprecationWarning: DataFram

layout      benchmark size  gate  depth  \
layout                                                                
full_10_2      15        full_10_2            ghz    5     7      7   
               2         full_10_2             dj    5    36     11   
               28        full_10_2     graphstate    5    50     22   
               80        full_10_2            qft    5    71     38   
               184       full_10_2         wstate    5    73     45   
...                            ...            ...  ...   ...    ...   
t_vertical_5_4 152  t_vertical_5_4      su2random    5   150     41   
               113  t_vertical_5_4            qnn    5   154     58   
               48   t_vertical_5_4  portfolioqaoa    5   195     72   
               126  t_vertical_5_4         random    5   195    117   
               61   t_vertical_5_4   portfoliovqe    5   310    107   

                    swap basic  swap sabre swap lookahead swap delta basic  \
layout                                                                       
full_10_2      15            0           0              0              nan   
               2             0           3              0              nan   
               28            0           0              0              nan   
               80            0           0              0              nan   
               184           0           0              0              nan   
...                        ...         ...            ...              ...   
t_vertical_5_4 152         117          48             60            48.72   
               113          81          48             66            18.52   
               48          117          51             87            25.64   
               126          36          12             66           -83.33   
               61          117          48             93            20.51   

                   swap delta sabre  depth basic  depth sabre depth lookahead  \
layout                                                                          
full_10_2      15               nan            7            7               7   
               2                100           11           17              11   
               28               nan           22           22              22   
               80               nan           38           38              38   
               184              nan           45           45              45   
...                             ...          ...          ...             ...   
t_vertical_5_4 152              -25          198          110              70   
               113            -37.5          172          127              84   
               48            -70.59          252          164             110   
               126             -450          151          106             121   
               61            -93.75          239          193             125   

                   depth delta basic depth delta sabre  
layout                                                  
full_10_2      15                  0                 0  
               2                   0             35.29  
               28                  0                 0  
               80                  0                 0  
               184                 0                 0  
...                              ...               ...  
t_vertical_5_4 152             64.65             36.36  
               113             51.16             33.86  
               48              56.35             32.93  
               126             19.87            -14.15  
               61               47.7             35.23  

[195 rows x 15 columns]

In [25]:
print("Benchmark [5]\n", df_benchmark_5.value_counts("benchmark"))
print("Benchmark [10]\n", df_benchmark_10.value_counts("benchmark"))
print("Benchmark [15]\n", df_benchmark_15.value_counts("benchmark"))

Benchmark [5]
 benchmark
dj                13
ghz               13
graphstate        13
portfolioqaoa     13
portfoliovqe      13
qaoa              13
qft               13
qftentangled      13
qnn               13
random            13
realamprandom     13
su2random         13
twolocalrandom    13
vqe               13
wstate            13
Name: count, dtype: int64
Benchmark [10]
 benchmark
dj                13
ghz               13
graphstate        13
portfolioqaoa     13
portfoliovqe      13
qaoa              13
qft               13
qftentangled      13
qnn               13
random            13
realamprandom     13
su2random         13
twolocalrandom    13
vqe               13
wstate            13
Name: count, dtype: int64
Benchmark [15]
 benchmark
dj                13
ghz               13
graphstate        13
portfolioqaoa     13
portfoliovqe      13
qaoa              13
qft               13
qftentangled      13
qnn               13
random            13
realamprandom     13
su2random 

In [26]:
df_layout_15

layout      benchmark     g    d  $s_B$  $s_S$  \
layout                                                                          
full_10_2      405        full\_10\_2            ghz    17   17      0     12   
               392        full\_10\_2             dj   118   22     66      6   
               418        full\_10\_2     graphstate   150   31     18      3   
               561        full\_10\_2            vqe   253   31      0      6   
               574        full\_10\_2         wstate   253  135      0      6   
...                               ...            ...   ...  ...    ...    ...   
t_vertical_5_4 542  t\_vertical\_5\_4      su2random   675   81   5304   1041   
               502  t\_vertical\_5\_4            qnn   914  158   3669    618   
               437  t\_vertical\_5\_4  portfolioqaoa  1260  192   5304    822   
               451  t\_vertical\_5\_4   portfoliovqe  2505  327   5304   1014   
               515  t\_vertical\_5\_4         random  2542  581   2475   1239   

                   $s_L$ $\Delta s_B$ $\Delta s_S$  $d_B$  $d_S$ $d_L$  \
layout                                                                   
full_10_2      405     0          nan          100     17     23    17   
               392     9        86.36          -50     95     27    29   
               418    15        16.67         -400     57     26    29   
               561     0          nan          100     31     40    31   
               574     0          nan          100    135    138   135   
...                  ...          ...          ...    ...    ...   ...   
t_vertical_5_4 542  1098         79.3        -5.48   1962    604   265   
               502  1077        70.65       -74.27   1449    547   344   
               437  1440        72.85       -75.18   2150    664   430   
               451  1251        76.41       -23.37   2280    881   456   
               515  1800        27.27       -45.28   2366   1243   658   

                   $\Delta d_B$ $\Delta d_S$  
layout                                        
full_10_2      405            0        26.09  
               392        69.47        -7.41  
               418        49.12       -11.54  
               561            0         22.5  
               574            0         2.17  
...                         ...          ...  
t_vertical_5_4 542        86.49        56.13  
               502        76.26        37.11  
               437           80        35.24  
               451           80        48.24  
               515        72.19        47.06  

[195 rows x 14 columns]